# INFO 300 Final Project

#### Metric Ideas:
- Youtube “Interaction’
    - Function of likes, dislikes, comments (possibly take into account comments/ratings_disabled)
    - Measures how much interaction users had with the content
- Views
- Category (get mapping through XX_category_id.json files)

- Time
    - Trending date
        - Talk about this (our data is already “trending data”, so we’d be looking at what trends most and least amongst already trending videos, not all videos) —> do we want more data?

#### Northern Meteorological Seasons
- Spring: March 1 to May 31
- Summer: June 1 to August 31
- Fall: September 1 to November 30
- Winter: December 1 to February 28 (February 29 on a leap year)

In [1]:
# Import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Data Preprocessing

In [2]:
# Load all country datasets into DataFrames
US = pd.read_csv('./../youtube-new/USvideos.csv')
CA = pd.read_csv('./../youtube-new/CAvideos.csv', encoding='UTF-8')
DE = pd.read_csv('./../youtube-new/DEvideos.csv', encoding='UTF-8')
FR = pd.read_csv('./../youtube-new/FRvideos.csv', encoding='UTF-8')
GB = pd.read_csv('./../youtube-new/GBvideos.csv', encoding='UTF-8')
IN = pd.read_csv('./../youtube-new/INvideos.csv', encoding='UTF-8')

# Encoding issues with these 4 countries (non-language characters appear, is unreadable)
# Excluding these countries from data analysis for now
JP = pd.read_csv('./../youtube-new/JPvideos.csv', encoding='ISO-8859-1')
KR = pd.read_csv('./../youtube-new/KRvideos.csv', encoding='ISO-8859-1')
MX = pd.read_csv('./../youtube-new/MXvideos.csv', encoding='ISO-8859-1')
RU = pd.read_csv('./../youtube-new/RUvideos.csv', encoding='ISO-8859-1')

all_countries = ['US', 'CA', 'DE', 'FR', 'GB', 'IN', 'JP', 'KR', 'MX', 'RU']
badly_encoded = ['JP', 'KR', 'MX', 'RU']
country_names = all_countries[0:-len(badly_encoded)]
country_data = [US, CA, DE, FR, GB, IN]

# Below snippet of code prints out the encoding found for the given file.
# for country in country_names:
#     filename = './../youtube-new/' + country + 'videos.csv'
#     with open(filename) as f:
#         print(f)

In [5]:
# Print out how many records and features each dataset has
for i, country in enumerate(country_data):
    print("The " + country_names[i] + " dataset contains " + str(country.shape[0]) + " entries and " + str(country.shape[1]) + " features.")

# Combine all countries into one dataframe
df = pd.DataFrame().append(country_data, ignore_index=True)
print("Our entire dataset contains " + str(df.shape[0]) + " entries and " + str(df.shape[1]) + " features.")

The US dataset contains 40949 entries and 16 features.
The CA dataset contains 40881 entries and 16 features.
The DE dataset contains 40840 entries and 16 features.
The FR dataset contains 40724 entries and 16 features.
The GB dataset contains 38916 entries and 16 features.
The IN dataset contains 37352 entries and 16 features.
Our entire dataset contains 239662 entries and 16 features.


In [27]:
print("The first video was published " + str(df['publish_time'].min()) + " and the last video was published " + str(df['publish_time'].max()))

trending_year_counts = {}
for row in df['trending_date']:
    year = int('20' + row[0:2])  # get year
    trending_year_counts[year] = trending_year_counts.get(year, 0) + 1

published_year_counts = {}
for row in df['publish_time']:
    year = int(row[0:4])  # get year
    published_year_counts[year] = published_year_counts.get(year, 0) + 1

print("\nNumber of videos trending: ")
trending_year_counts_df = pd.DataFrame(data=list(trending_year_counts.items()), columns=['Date', 'Trending Count']).sort_values(by='Date')
print(trending_year_counts_df)

print("\nNumber of videos published: ")
published_year_counts_df = pd.DataFrame(data=list(published_year_counts.items()), columns=['Date', 'Published Count']).sort_values(by='Date')
print(published_year_counts_df)

The first video was published 2006-07-23T08:24:11.000Z and the last video was published 2018-06-14T02:25:38.000Z

Number of videos trending: 
   Date  Trending Count
0  2017           57037
1  2018          182625

Number of videos published: 
    Date  Published Count
11  2006                1
12  2007               37
8   2008               59
6   2009               31
4   2010               57
1   2011              113
3   2012               41
7   2013              122
9   2014              114
2   2015              143
5   2016              127
0   2017            60600
10  2018           178217


In [28]:
# Convert date strings in data to actual timestamp objects to make date querying cleaner
import datetime

print("Before:")
print(type(df['trending_date'][0]))
print(df['trending_date'][0])

df['trending_date'] = pd.to_datetime(df['trending_date'], format="%y.%d.%m")
df['publish_time'] = pd.to_datetime(df['publish_time'], yearfirst=True, infer_datetime_format=True, utc=True)

print("\nAfter:")
print(type(df['publish_time'][0]))
print(df['publish_time'][0])

Before:
<class 'str'>
17.14.11

After:
<class 'pandas._libs.tslib.Timestamp'>
2017-11-13 17:13:01+00:00


In [29]:
# Video categories for all countries are the same, so only need to use US's categories
json = pd.read_json('./../youtube-new/US_category_id.json')

# Get video category id and title. id: video_categories[0], title: video_categories[1]
video_categories = {}
categories = [[item['id'], item['snippet']['title']] for item in json['items']]
for category in categories:
    id = int(category[0])
    name = category[1]
    video_categories[id] = name

print(video_categories)

{1: 'Film & Animation', 2: 'Autos & Vehicles', 10: 'Music', 15: 'Pets & Animals', 17: 'Sports', 18: 'Short Movies', 19: 'Travel & Events', 20: 'Gaming', 21: 'Videoblogging', 22: 'People & Blogs', 23: 'Comedy', 24: 'Entertainment', 25: 'News & Politics', 26: 'Howto & Style', 27: 'Education', 28: 'Science & Technology', 29: 'Nonprofits & Activism', 30: 'Movies', 31: 'Anime/Animation', 32: 'Action/Adventure', 33: 'Classics', 34: 'Comedy', 35: 'Documentary', 36: 'Drama', 37: 'Family', 38: 'Foreign', 39: 'Horror', 40: 'Sci-Fi/Fantasy', 41: 'Thriller', 42: 'Shorts', 43: 'Shows', 44: 'Trailers'}


## Data Analysis

### Select Features and Filter Data

In [30]:
basic_features = ['views', 'likes', 'dislikes', 'comment_count']
exclude_features = ['thumbnail_link', 'comments_disabled', 'ratings_disabled', 'video_error_or_removed', 'description']

num_records_total = len(df)

# Filter out videos that have comments/ratings disabled or got removed
# and drop all excluded feature columns
df = df[(df[exclude_features[1]] == False) & 
        (df[exclude_features[2]] == False) &
        (df[exclude_features[3]] == False)].drop(columns=exclude_features)

# 696 records filtered. We could have used this intersection of data for further analysis
# but a sample size of ~700 out of ~41k might be too small to yield interesting findings
num_records_filtered = num_records_total - len(df)

# Preview
df.iloc[0:1]

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count
0,2kyS6SvSYSE,2017-11-14,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13 17:13:01+00:00,SHANtell martin,748374,57527,2966,15954


### Normalizing features

In [31]:
# Normalize all feature columns
def normalize(col):
    largest = col.max()
    if largest == 0:
        return col
    return col / largest

for feature in basic_features:
    df[feature] = normalize(df[feature])
    
# Normalized preview
df[['title'] + basic_features][0:5]

,title,views,likes,dislikes,comment_count
0,WE WANT TO TALK ABOUT OUR MARRIAGE,0.001763,0.010247,0.001525,0.009809
1,The Trump Presidency: Last Week Tonight with J...,0.005697,0.017312,0.003160,0.007810
2,"Racist Superman | Rudy Mancuso, King Bach & Le...",0.007517,0.026013,0.002745,0.005030
3,Nickelback Lyrics: Real or Fake?,0.000808,0.001812,0.000342,0.001319
4,I Dare You: GOING BALD!?,0.004936,0.023555,0.001023,0.010770


### Creating Metrics

In [39]:
# Split by season
spring_mask = df['trending_date'].map(lambda x: x.month) == 3
spring = df[spring_mask]
spring
# spring = df[df['trending_date'].dt.month.str.contains('0[345]$')] # 03, 04, 05
# summer = df[df['trending_date'].str.contains('0[678]$')] # 06, 07, 08
# fall = df[df['trending_date'].str.contains('09$|1[01]$')] # 09, 10, 11
# winter = df[df['trending_date'].str.contains('12$|0[12]$')] # 12, 01, 02

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count
20967,DIBw9dSVKdU,2018-03-01,Ralph Breaks The Internet: Wreck-It Ralph 2 Of...,Walt Disney Animation Studios,1,2018-02-28 13:43:42+00:00,"Disney|""Walt Disney Animation Studios""|""Disney...",0.009949,0.023100,0.002054,0.016728
20968,IJofEhiRpy0,2018-03-01,I WORE MY BOYFRIEND'S CLOTHES FOR A WEEK!,Ryland Adams,24,2018-02-28 20:00:00+00:00,"Ryland Adams|""Shane Dawson""|""Boyfriend""|""Cloth...",0.002412,0.014243,0.000701,0.003409
20969,T3hXC5yNZOc,2018-03-01,Iggy Azalea - Savior ft. Quavo,iggyazaleamusicVEVO,10,2018-03-01 05:00:01+00:00,"Iggy|""Azalea""|""Savior""|""Island""|""Records""|""Hip...",0.000674,0.006079,0.000835,0.002594
20970,expKmfdoo28,2018-03-01,Chadwick Boseman Surprises Black Panther Fans ...,The Tonight Show Starring Jimmy Fallon,23,2018-03-01 04:46:38+00:00,"The Tonight Show|""Jimmy Fallon""|""Chadwick Bose...",0.000229,0.001215,0.000086,0.000381
20971,Kf2-86o5S1o,2018-03-01,Film Theory: The Bee Movie LIED To You!,The Film Theorists,1,2018-02-27 22:07:31+00:00,"bees|""honey bee""|""bee movie""|""the bee movie""|""...",0.007613,0.019467,0.003164,0.011826
20972,5tPugvYE6wM,2018-03-01,KYLIE COSMETICS WEATHER COLLECTION REVIEW + TU...,Nicol Concilio,26,2018-02-28 21:00:35+00:00,"kylie cosmetics|""kylie cosmetics weather""|""kyl...",0.000498,0.001957,0.000429,0.001195
20973,kOsl3cmK3zg,2018-03-01,Google Search Team Answers the Web's Most Sear...,WIRED,24,2018-02-28 17:00:33+00:00,"autocomplete|""google autocomplete""|""google aut...",0.000235,0.000842,0.000078,0.000229
20974,n9JVbbRBqfY,2018-03-01,LEGENDARY All You Can Eat Buffet in Manila Phi...,Strictly Dumpling,19,2018-02-28 02:13:58+00:00,"spiral buffet|""all you can eat""|""best buffet""|...",0.001605,0.003745,0.000181,0.001828
20975,ZzO8VAOJs4U,2018-03-01,Kiefer Sutherland's Mom Has An Impersonation O...,The Late Show with Stephen Colbert,24,2018-02-28 08:35:00+00:00,"The Late Show|""Stephen Colbert""|""Colbert""|""Lat...",0.000690,0.000453,0.000035,0.000223
20976,3tjHS7JRB4c,2018-03-01,GIANT Pie In The Face!!,Team Edge,23,2018-02-27 21:00:01+00:00,"challenges|""competition""|""family fun""|""kid fri...",0.002382,0.003983,0.000252,0.002965


In [8]:
# Function that retrieves the title of the the video with the *maximum* value for a given feature in a season
def get_max_metric(feature, season):
    season_df = globals()[season]  
    max_metric = season_df[season_df[feature] == season_df[feature].max()]
    return max_metric['title'].to_string(index=False)

# Function that retrieves the title of the the video with the *minimum* value for a given feature in a season
def get_min_metric(feature, season):
    season_df = globals()[season]  
    min_metric = season_df[season_df[feature] == season_df[feature].min()]
    return min_metric['title'].to_string(index=False)

# Views Metric
print("The video(s) with the most views is %s" %(get_max_metric('views', 'spring')))
print("The video(s) with the fewest views is %s" %(get_min_metric('views', 'spring')))
print()

# Likes Metric
print("The video(s) with the most likes is %s" %(get_max_metric('likes', 'spring')))
print("The video(s) with the fewest likes is %s" %(get_min_metric('likes', 'spring')))
print()

# Dislikes Metric
print("The video(s) with the most dislikes is %s" %(get_max_metric('dislikes', 'spring')))
print("The video(s) with the fewest dislikes is %s" %(get_min_metric('dislikes', 'spring')))
print()

# Comment Count Metric
print("The video(s) with the most comments is %s" %(get_max_metric('comment_count', 'spring')))
print("The video(s) with the fewest comments is %s" %(get_min_metric('comment_count', 'spring')))
print()

# Combined Metric



The video(s) with the most views is Childish Gambino - This Is America (Official V...
The video(s) with the fewest views is President Trump set to announce 2020 re-electi...

The video(s) with the most likes is BTS (방탄소년단) 'FAKE LOVE' Official MV
The video(s) with the fewest likes is President Trump set to announce 2020 re-electi...

The video(s) with the most dislikes is Childish Gambino - This Is America (Official V...
The video(s) with the fewest dislikes is Bird Lands On News Anchor's Head (News Blooper)
  Rescued Chimp Helps Out on Flight over Africa

The video(s) with the most comments is BTS (방탄소년단) 'FAKE LOVE' Official MV
The video(s) with the fewest comments is Bird Lands On News Anchor's Head (News Blooper)



In [9]:
# Double checking that there's actually a tie for fewest dislikes in spring

print("The video with the fewest dislikes is %s" %(get_min_metric('dislikes', 'spring')))
print()

spring[spring['dislikes'] == spring['dislikes'].min()]['title']
# spring[0]'
print(spring[spring['title'] == "Bird Lands On News Anchor's Head (News Blooper)"][basic_features])
print(spring[spring['title'] == "Rescued Chimp Helps Out on Flight over Africa"][basic_features])

The video with the fewest dislikes is Bird Lands On News Anchor's Head (News Blooper)
  Rescued Chimp Helps Out on Flight over Africa

          views     likes  dislikes  comment_count
21125  0.000038  0.000013  0.000002       0.000006
          views     likes  dislikes  comment_count
21640  0.000082  0.000036  0.000002       0.000012
21841  0.000211  0.000090  0.000007       0.000053
22063  0.000286  0.000121  0.000008       0.000073


In [10]:
# Min and Max metrics for all basic features and all seasons
seasons = ['spring', 'summer', 'fall', 'winter']

def print_min_and_max_metrics(feature, season):
    print("The video(s) with the most %s is %s" %(feature, get_max_metric(feature, season)))
    print("The video(s) with the fewest %s is %s" %(feature, get_min_metric(feature, season)))
    print()

for season in seasons:
    for feature in basic_features:
        print_min_and_max_metrics(feature, season)

The video(s) with the most views is Childish Gambino - This Is America (Official V...
The video(s) with the fewest views is President Trump set to announce 2020 re-electi...

The video(s) with the most likes is BTS (방탄소년단) 'FAKE LOVE' Official MV
The video(s) with the fewest likes is President Trump set to announce 2020 re-electi...

The video(s) with the most dislikes is Childish Gambino - This Is America (Official V...
The video(s) with the fewest dislikes is Bird Lands On News Anchor's Head (News Blooper)
  Rescued Chimp Helps Out on Flight over Africa

The video(s) with the most comment_count is BTS (방탄소년단) 'FAKE LOVE' Official MV
The video(s) with the fewest comment_count is Bird Lands On News Anchor's Head (News Blooper)

The video(s) with the most views is Childish Gambino - This Is America (Official V...
The video(s) with the fewest views is Josh Groban - Granted (Official Lyric Video)

The video(s) with the most likes is BTS (방탄소년단) 'FAKE LOVE' Official MV
The video(s) with th

In [17]:
# Function to get the category name of a record
def category(record):
    return video_categories[record['category_id']]

category_groups = df.groupby('category_id').groups



for season in seasons:
    for feature in basic_features:
        plt.plot(get_max_metric(feature, season), '.')

1
1


In [144]:
import dateutil
snippet = df[0:20].copy()
print(snippet[0:1]['publish_time'])
prev_date = snippet.iloc[0]['publish_time']
print(type(prev_date))
# dateutil.parser.parse?
snippet['publish_time'] = snippet['publish_time'].apply(dateutil.parser.parse, yearfirst=True)
print(snippet[0:1]['publish_time'])
post_date = snippet.iloc[0]['publish_time']
print(type(post_date))
# df.iloc[0]['publish_time']
print()
print(post_date.year)

0    2017-11-13T17:13:01.000Z
Name: publish_time, dtype: object
<class 'str'>
0   2017-11-13 17:13:01+00:00
Name: publish_time, dtype: datetime64[ns, UTC]
<class 'pandas._libs.tslib.Timestamp'>

2017


In [ ]:
for country in country_names

In [133]:
category_groups = df.groupby('category_id').groups


{1: Int64Index([     8,     29,     36,     71,     86,     94,    108,    169,
                210,    218,
             ...
             239494, 239499, 239512, 239516, 239585, 239612, 239616, 239621,
             239634, 239638],
            dtype='int64', length=13173),
 2: Int64Index([    30,     58,    163,    197,    265,    273,    302,    496,
                503,    531,
             ...
             228807, 229322, 229348, 230203, 230341, 230454, 231037, 231259,
             233255, 233468],
            dtype='int64', length=2499),
 10: Int64Index([    12,     32,     37,     39,     40,     43,     53,     63,
                 70,     74,
             ...
             239575, 239576, 239582, 239604, 239607, 239620, 239623, 239632,
             239633, 239639],
            dtype='int64', length=34133),
 15: Int64Index([    14,     92,     97,    128,    167,    270,    352,    353,
                386,    463,
             ...
             201628, 201756, 201794, 201927, 201

'People & Blogs'